In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter

# pd.set_option('max_colwidth', 100)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
def clear_iri(text):
    match = re.findall('#.*', text)
    if match:
        res = match[0].replace('"', '').replace('#', '')
        return res

def take_needed_cols_Substations(path, cols_from_modeller, cols2preparator, id2check=None):
    df = pd.read_csv(path, sep=';')
    
    
    assert len(cols_from_modeller) == len(cols2preparator)
    
    df['Класс'] = 'Substation'
    for col in ['name', 'Region']:
        df[col] = df[col].apply(lambda x: x.replace('"', '').replace('=', '') if isinstance(x, str) else '')
    
    df['IRI'] = df['IRI'].apply(clear_iri)
    
    mod2prep = {key:val for key, val in zip(cols_from_modeller, cols2preparator)}
    df = df[cols_from_modeller]
    df = df.rename(columns=mod2prep)
    df = df.drop_duplicates()
    if id2check:
        assert df[id2check].duplicated().sum() == 0, 'ther are some duplicates in ids'
    return df

In [3]:
path = 'data_from_modeller/Подстанция_2023-12-16.csv'
cols_from_modeller = ['IRI', 'name', 'Класс', 'Region']
cols2preparator = ['Uid', 'name', 'Класс', 'Substations']


df_rs = take_needed_cols_Substations(path, cols_from_modeller, cols2preparator, id2check='Uid')
df_rs = df_rs[df_rs['name'].str.contains('110')]

# ручное
df_rs.loc[df_rs['name'] == 'ПС 110 кВ Баженово', 'Substations'] = 'Омская область'
df_rs.loc[df_rs['name'] == 'ПС 110 кВ Береговая (МЭС)', 'name'] = 'ПС 110 кВ БереговаяМЭС'
df_rs.loc[df_rs['Uid'] == '5ea93f73-4576-4cf8-8693-4a2b87c83816', 'Substations'] = 'Омская область'
df_rs.loc[df_rs['name'] == 'ПС 110 кВ Городская (Красноярск)', 'name'] = 'Городская-кэс'
df_rs.loc[df_rs['name'] == 'ПС 110 кВ Городская (ВЭС)', 'name'] = 'Городская-вэс'
df_rs.loc[df_rs['name'] == 'ПС 110 кВ Городская (СЭС)', 'name'] = 'Городская-сэс'
df_rs.loc[df_rs['Uid'] == '985a1a39-ba35-4baa-8647-d597ad014e40', 'Substations'] = 'Омская область'
df_rs.loc[df_rs['Uid'] == '66a545f3-f31b-4431-b970-8f41493b5e45', 'Substations'] = 'Омская область'
df_rs.loc[df_rs['name'] == 'ПС 110 кВ ТПК Надеждинский', 'Substations'] = 'Омская область'
# ---

display(df_rs.head())
df_rs.info()
for col in df_rs.columns:
    print(col, df_rs[col].duplicated().sum())

df_rs.to_csv('output_data/Substation/Substation4preparator_full.csv', index=False)

,Uid,name,Класс,Substations
0,000fe64c-7b85-4bea-9ada-3e514d968485,ПС 110 кВ Уропская,Substation,Кемеровская область
2,011f18d7-614e-4945-8bf8-81964a8c6a9f,ПС 110 кВ Потаскуй,Substation,Красноярский край
3,01628ca3-8af1-4d26-84c3-52ee6c0f1adc,ПС 110 кВ Пригородная,Substation,Алтайский край
4,018c4cfb-1f78-41e9-8e40-001a41114299,ПС 110 кВ Городская,Substation,Кемеровская область
5,0193398d-8d84-40d9-8520-2f8ea56bf5e9,ПС 110 кВ Коробейниково,Substation,Алтайский край


<class 'pandas.core.frame.DataFrame'>
Index: 836 entries, 0 to 851
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Uid          836 non-null    object
 1   name         836 non-null    object
 2   Класс        836 non-null    object
 3   Substations  836 non-null    object
dtypes: object(4)
memory usage: 32.7+ KB
Uid 0
name 51
Класс 835
Substations 827


In [4]:
be2region = {
    4200: 'Кемеровская область',
    2400: 'Красноярский край',
    2200: 'Алтайский край',
    300: 'Республика Бурятия',
    1900: 'Республика Хакасия',
    7500: 'Забайкальский край',
    400: 'Республика Алтай',
    5500: 'Омская область',
}

In [5]:
df_supa = pd.read_excel('data_supa/ТМ_ПС_РП_20231221.xlsx')
df_supa['БЕ'] = df_supa['БЕ'].astype('int')
df_supa['region'] = df_supa['БЕ'].apply(lambda x: be2region.get(x))
df_supa = df_supa.dropna(subset='region')

# ручное
df_supa.loc[df_supa['Название'] == 'ПС 110 кВ Береговая (МЭС)', 'Название'] = 'ПС 110 кВ БереговаяМЭС'
df_supa.loc[df_supa['Дисп.наименование'] == 'ПС 110 кВ Городская (КЭС)', 'Название'] = 'Городская-кэс'
df_supa.loc[df_supa['Дисп.наименование'] == 'ПС 110 кВ Городская (ВЭС)', 'Название'] = 'Городская-вэс'
df_supa.loc[df_supa['Дисп.наименование'] == 'ПС 110 кВ Городская (СЭС)', 'Название'] = 'Городская-сэс'
df_supa.loc[df_supa['Дисп.наименование'] == 'ПС 110 кВ Забайкальск тяговая', 'Название'] = 'ПС 110 кВ Забайкальск тяговая'
# ---

display(df_supa.head())
display(df_supa.info())
for col in df_supa.columns:
    print(col, df_supa[col].duplicated().sum())

,ГруппПолномочий,Техместо,Название,Дисп.наименование,Местоположение,НаименованиеМестоположения,ВерхнееТМ,Район,Класс,БЕ,...,Видобъекта,ОснСредство,ПользовСтатус,СистемнСтатус,Вэксплуат.с,Годвыпуска,Инвентарн№,НомерОСверх.ТМ,НомерОСинв.об.,region
0,754B,PS110-000002,ПС 110 кВ Улеты,ПС 110 кВ Улеты,7540020200,Улетовский РЭС,PS110-000002,NaN,PS,7500,...,1023,T7500SRSK,ДСТВ,СОЗД,1968-10-01 00:00:00,NaN,NaN,T7500SRSK,NaN,Забайкальский край
1,2230,PS110-000004,"ПС ""Гидроузел"" №29",ПС 110 кВ Гидроузел,2230000000,Западные Э/С,PS110-000004,NaN,PS,2200,...,1023,106012110,ДСТВ,СОЗД,1977-06-01 00:00:00,1977.0,Б000023634,106012110,NaN,Алтайский край
2,192A,PS110-000005,ПС 110 кВ Первомайская,ПС 110 кВ Первомайская,1920020100,Боградский РЭС,PS110-000005,NaN,PS,1900,...,1023,106000406,ДСТВ,СОЗД,1990-08-14 00:00:00,NaN,К000009305,106000406,NaN,Республика Хакасия
3,031B,PS110-000006,"ПС 110/35/6-10 кВ ""Тимлюйская""",ПС 110 кВ Тимлюйская,310020200,Кабанский РЭС,PS110-000006,NaN,PS,300,...,1021,106000846,ДСТВ,СОЗД,1966-01-01 00:00:00,1966.0,В50010722,106000846,NaN,Республика Бурятия
4,041D,PS110-000007,ПС 110 кВ Турочакская №18,ПС 110кВ Турачакская,410020400,Турачакский РЭС,PS110-000007,NaN,PS,400,...,1023,106000613,ДСТВ,СОЗД,1986-06-01 00:00:00,NaN,М000002929,106000613,NaN,Республика Алтай


<class 'pandas.core.frame.DataFrame'>
Index: 1188 entries, 0 to 1205
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ГруппПолномочий             1188 non-null   object 
 1   Техместо                    1188 non-null   object 
 2   Название                    1188 non-null   object 
 3   Дисп.наименование           1149 non-null   object 
 4   Местоположение              1188 non-null   int64  
 5   НаименованиеМестоположения  1188 non-null   object 
 6   ВерхнееТМ                   1188 non-null   object 
 7   Район                       149 non-null    object 
 8   Класс                       1188 non-null   object 
 9   БЕ                          1188 non-null   int64  
 10  ПланирующЗавод              1188 non-null   int64  
 11  Видобъекта                  1188 non-null   int64  
 12  ОснСредство                 1186 non-null   object 
 13  ПользовСтатус               1188 non-n

None

ГруппПолномочий 990
Техместо 0
Название 16
Дисп.наименование 115
Местоположение 995
НаименованиеМестоположения 1000
ВерхнееТМ 0
Район 1130
Класс 1187
БЕ 1180
ПланирующЗавод 1154
Видобъекта 1181
ОснСредство 699
ПользовСтатус 1181
СистемнСтатус 1187
Вэксплуат.с 855
Годвыпуска 1109
Инвентарн№ 671
НомерОСверх.ТМ 699
НомерОСинв.об. 1187
region 1180


In [6]:
# df_supa['Название'].unique().tolist()
# df_rs['name'].unique().tolist()

# name + region

In [7]:
texts = [
    'ПС 110/35/10 кВ "Тарбагатай"',
    'ПС "Георгиевская"',
    'ПС "Удинская"',
    'ПС "Хоринская"',
    'ПС "Бичура"',
    'ПС "Окино-Ключи"',
    'ПС "Тухум"',
    'ПС "Холбольджино"',
    'ПС "Боргой"',
    'ПС "Бургултай"',
    'ПС "Джида"',
    'ПС "Петропавловка"',
    'ПС "Торей"',
    'ПС "Инкурская"',
    'ПС "Большой Луг"',
    'ПС "Кудара Самон"',
    'ПС "Кяхта"',
    'ПС "Харанхой"',
    'ПС "Никольская"',
    'ПС "Подлопатки"',
    'ПС 110 кВ Кош-Агачская №37',
    'ПС 110 кВ Ининская №25',
    'ПС 110 кВ Онгудайская №24',
    'ПС 110 кВ Новолюбинская',
    'ПС 110 кВ Верхняя Давенда',
    'ПС 110 кВ Тагарская',
    'ПС 110 кВ Вершина Шахтамы (110/6)',
    'ПС "Кормиловка Т" 110/10',
    'ПС 110 кВ Назаровский ПП',
    'ПС 110 кВ Весна-2',
    'ПС "Баженово-110" 110/35/10',
    'ПС 110 кВ Вознесенка',
    'ПС 110 кВ Заозёрновская'
]

text2num = {
    '1': 'один',
    '2': 'два',
    '3': 'три',
    '4': 'четыре',
    '5': 'пять',
    '6': 'шесть',
    '7': 'семь',
    '8': 'восемь',
    '9': 'девять'
}

def get_num(text):
    match = re.findall('[а-яА-Я]+-?\d', text)
    if match:
        num = re.findall('\d', match[0])[0]
        return re.sub('\d', f'{text2num[num]}', match[0])
    return text
        

def get_name(text):
    text = text.replace('Н Бирилюссы', 'Новобирилюссы')
    text = text.replace('Б.Мурта', 'Большая Мурта')
    text = text.replace('Б.Салба', 'Большая Салба')
    text = text.replace('Самон', 'Сомон')
    text = text.replace('Кебезеньская', 'Кебезенская')
    text = text.replace('Валуевская ', 'Валуевская-')
    text = text.replace('Насосная станция оборотной воды', 'Насосная Низовая')
    text = text.replace('Ново-Енисейский', 'Новоенисейский')
    text = text.replace('Н.Енисейский', 'Новоенисейский')
    text = text.replace('Ново-Широкая', 'Новоширокая')
    text = text.replace('ПС "Октябрьская 2"', 'Октябрьская-2')
    text = text.replace('ПС 110 кВ Тепличная (Осинник. РЭС)', 'ПС 110 кВ Осинниковская Тепличная')
    text = text.replace('ПС "Улан-Удэнскптицефабрика (ПТФ)"', 'ПС Птицефабрика')
    text = text.replace('Северная (ЗЭС)', 'СевернаяЗЭС')
    text = text.replace('Сосново-Озерская', 'Сосновоозерская')
    text = text.replace('ПС "Стройбаза" С', 'ПС Стройбаза')
    text = text.replace('ПС "Надеждинский ТПК" 110/10', 'ПС 110 кВ ТПК Надеждинский')
    text = text.replace('Устькалманская', 'Усть-Калманская')
    text = text.replace('Цвет. мет', 'Цветметные металлы')
    text = text.replace('ПС 110 кВ', '')
    text = text.replace('ТПС', '')
    text = text.replace('НПС', '')
    text = text.replace('ПС', '')
    text = text.replace('ПП', '')
    text = text.replace('"', '')
    text = text.replace('№', '')
    text = text.replace('кВ', '')
    text = text.replace('110', '')
    
    text = get_num(text)
    match = re.findall('[А-Яа-яёЁ]+[ -]?[А-Яа-яёЁ]+', text)
    if match:
        res = match[0].replace('ская', '')
        res = re.sub('ка[$ ]?', ' ', res)
        res = res.replace('ё', 'е')
        res = re.sub('Т$', '', res)
        res = res.strip()
        res = res.replace(' ', '-')
        return res.lower()
    return ''

for text in texts:
    print(text, '->', get_name(text))

ПС 110/35/10 кВ "Тарбагатай" -> тарбагатай
ПС "Георгиевская" -> георгиев
ПС "Удинская" -> удин
ПС "Хоринская" -> хорин
ПС "Бичура" -> бичура
ПС "Окино-Ключи" -> окино-ключи
ПС "Тухум" -> тухум
ПС "Холбольджино" -> холбольджино
ПС "Боргой" -> боргой
ПС "Бургултай" -> бургултай
ПС "Джида" -> джида
ПС "Петропавловка" -> петропавлов
ПС "Торей" -> торей
ПС "Инкурская" -> инкур
ПС "Большой Луг" -> большой-луг
ПС "Кудара Самон" -> кудара-сомон
ПС "Кяхта" -> кяхта
ПС "Харанхой" -> харанхой
ПС "Никольская" -> николь
ПС "Подлопатки" -> подлопатки
ПС 110 кВ Кош-Агачская №37 -> кош-агач
ПС 110 кВ Ининская №25 -> инин
ПС 110 кВ Онгудайская №24 -> онгудай
ПС 110 кВ Новолюбинская -> новолюбин
ПС 110 кВ Верхняя Давенда -> верхняя-давенда
ПС 110 кВ Тагарская -> тагар
ПС 110 кВ Вершина Шахтамы (110/6) -> вершина-шахтамы
ПС "Кормиловка Т" 110/10 -> кормилов
ПС 110 кВ Назаровский ПП -> назаровский
ПС 110 кВ Весна-2 -> весна-два
ПС "Баженово-110" 110/35/10 -> баженово
ПС 110 кВ Вознесенка -> вознесен
ПС 11

In [8]:
df_rs['name_norm'] = df_rs['name'].apply(get_name)
df_supa['name_norm'] = df_supa['Название'].apply(get_name)

df_rs['name_region'] = df_rs['name_norm'] +'_'+ df_rs['Substations']
df_supa['name_region'] = df_supa['name_norm'] +'_'+ df_supa['region']

# ручное
df_rs.loc[(df_rs['name'].str.contains('ГПП-2 ЦБК')) & (df_rs['Substations'] == 'Красноярский край'), 'name_region'] = 'гпп-два-цбк Красноярский край'
df_supa.loc[(df_supa['Название'].str.contains('ГПП-2 ЦБК')) & (df_supa['region'] == 'Красноярский край'), 'name_region'] = 'гпп-два-цбк Красноярский край'
df_rs.loc[(df_rs['name'].str.contains('ГПП-2')) & (df_rs['Substations'] == 'Республика Хакасия'), 'name_region'] = 'гпп-два Республика Хакасия'
df_supa.loc[(df_supa['Название'].str.contains('ГПП-2')) & (df_supa['region'] == 'Республика Хакасия'), 'name_region'] = 'гпп-два Республика Хакасия'
df_rs.loc[(df_rs['name'] == 'ПС 110 кВ 827 Объект') & (df_rs['Substations'] == 'Забайкальский край'), 'name_region'] = '827-объект Забайкальский край'
df_supa.loc[(df_supa['Название'] == 'ПС 110 кВ Обьект 827') & (df_supa['region'] == 'Забайкальский край'), 'name_region'] = '827-объект Забайкальский край'
# ---

supa_name_region_list = df_supa['name_region'].tolist()
rs_name_region_list = df_rs['name_region'].tolist()

supa_name_region = set(supa_name_region_list)
rs_name_region = set(rs_name_region_list)

print('supa_name_region & rs_name_region =', len(supa_name_region & rs_name_region))

supa_name_region_cnt = dict(Counter(supa_name_region_list))
rs_name_region_cnt = dict(Counter(rs_name_region_list))

supa_name_region_single = set([key for key in supa_name_region_cnt.keys() if supa_name_region_cnt[key] == 1])
rs_name_region_single = set([key for key in rs_name_region_cnt.keys() if rs_name_region_cnt[key] == 1])
print('supa_name_region_single & rs_name_region_single', len(supa_name_region_single & rs_name_region_single))

supa_name_region & rs_name_region = 683
supa_name_region_single & rs_name_region_single 671


In [9]:
df_merge1 = df_rs[df_rs['name_region'].isin(rs_name_region_single)].merge(df_supa[df_supa['name_region'].isin(supa_name_region_single)], on='name_region', suffixes=['_rs', '_supa'])
df_merge1.shape[0]

671

In [10]:
for col in df_merge1.columns:
    print(col, df_merge1[col].duplicated().sum())
    assert df_merge1[col].duplicated().sum() == 0 if col in ['Uid', 'Техместо'] else True

Uid 0
name 35
Класс_rs 670
Substations 663
name_norm_rs 51
name_region 0
ГруппПолномочий 522
Техместо 0
Название 11
Дисп.наименование 38
Местоположение 524
НаименованиеМестоположения 527
ВерхнееТМ 0
Район 614
Класс_supa 670
БЕ 663
ПланирующЗавод 643
Видобъекта 667
ОснСредство 320
ПользовСтатус 669
СистемнСтатус 670
Вэксплуат.с 401
Годвыпуска 597
Инвентарн№ 294
НомерОСверх.ТМ 320
НомерОСинв.об. 670
region 663
name_norm_supa 51


In [11]:
df_supa = df_supa[~df_supa['Техместо'].isin(df_merge1['Техместо'].unique())]
df_rs = df_rs[~df_rs['Uid'].isin(df_merge1['Uid'].unique())]
print(df_supa.shape[0], df_rs.shape[0])

517 165


# name_norm only

In [12]:
supa_name_norm_list = df_supa['name_norm'].tolist()
rs_name_norm_list = df_rs['name_norm'].tolist()

supa_name_norm = set(supa_name_norm_list)
rs_name_norm = set(rs_name_norm_list)

print('supa_name_norm & rs_name_norm =', len(supa_name_norm & rs_name_norm))

supa_name_norm_cnt = dict(Counter(supa_name_norm_list))
rs_name_norm_cnt = dict(Counter(rs_name_norm_list))

supa_name_norm_single = set([key for key in supa_name_norm_cnt.keys() if supa_name_norm_cnt[key] == 1])
rs_name_norm_single = set([key for key in rs_name_norm_cnt.keys() if rs_name_norm_cnt[key] == 1])
print('supa_name_norm_single & rs_name_norm_single', len(supa_name_norm_single & rs_name_norm_single))

supa_name_norm & rs_name_norm = 130
supa_name_norm_single & rs_name_norm_single 110


In [13]:
df_merge2 = df_rs[df_rs['name_norm'].isin(rs_name_norm_single)].merge(df_supa[df_supa['name_norm'].isin(supa_name_norm_single)], on='name_norm', suffixes=['_rs', '_supa'])
df_merge2.shape[0]

110

In [14]:
for col in df_merge2.columns:
    print(col, df_merge2[col].duplicated().sum())
    assert df_merge2[col].duplicated().sum() == 0 if col in ['Uid', 'Техместо'] else True

Uid 0
name 0
Класс_rs 109
Substations 109
name_norm 0
name_region_rs 0
ГруппПолномочий 79
Техместо 0
Название 0
Дисп.наименование 0
Местоположение 79
НаименованиеМестоположения 79
ВерхнееТМ 0
Район 104
Класс_supa 109
БЕ 109
ПланирующЗавод 107
Видобъекта 108
ОснСредство 7
ПользовСтатус 109
СистемнСтатус 109
Вэксплуат.с 50
Годвыпуска 73
Инвентарн№ 6
НомерОСверх.ТМ 7
НомерОСинв.об. 109
region 109
name_region_supa 0


In [15]:
df_supa = df_supa[~df_supa['Техместо'].isin(df_merge2['Техместо'].unique())]
df_rs = df_rs[~df_rs['Uid'].isin(df_merge2['Uid'].unique())]
print(df_supa.shape[0], df_rs.shape[0])

# filename = 'data_from_modeller/prepared/Substation4preparator_unmatched.csv'
# df_rs.to_csv(filename, index=False)
# print(f'test saved {filename}')
# display(pd.read_csv(filename).head())
# pd.read_csv(filename).info()

407 55


In [16]:
print((df_merge1.shape[0] + df_merge2.shape[0])/(df_supa.shape[0] + df_merge1.shape[0] + df_merge2.shape[0]))
(df_merge1.shape[0] + df_merge2.shape[0])/(df_rs.shape[0] + df_merge1.shape[0] + df_merge2.shape[0])

0.6574074074074074


0.9342105263157895

In [17]:
needed_cols = ['Uid', 'Техместо', 'name', 'Дисп.наименование', 'Класс_rs']
columns2rename = {'Класс_rs': 'Класс', 
                 'name': 'Наименование',
                 'Дисп.наименование': 'Диспетчерское наименование'}
df_union = pd.concat([df_merge1[needed_cols], df_merge2[needed_cols]]).rename(columns=columns2rename)
for col in df_union.columns:
    print(col, df_union[col].duplicated().sum())
print(df_union.shape)
display(df_union.info())
display(df_union.head())

# filename = 'output_data/Substation/Substation_result.csv'
# df_union.to_csv(filename, index=False)
# print(f'test saved {filename}')
# display(pd.read_csv(filename).head())
# pd.read_csv(filename).info()

Uid 0
Техместо 0
Наименование 44
Диспетчерское наименование 46
Класс 780
(781, 5)
<class 'pandas.core.frame.DataFrame'>
Index: 781 entries, 0 to 109
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Uid                         781 non-null    object
 1   Техместо                    781 non-null    object
 2   Наименование                781 non-null    object
 3   Диспетчерское наименование  778 non-null    object
 4   Класс                       781 non-null    object
dtypes: object(5)
memory usage: 36.6+ KB


None

,Uid,Техместо,Наименование,Диспетчерское наименование,Класс
0,000fe64c-7b85-4bea-9ada-3e514d968485,PS110-000516,ПС 110 кВ Уропская,ПС 110 кВ Уропская,Substation
1,011f18d7-614e-4945-8bf8-81964a8c6a9f,PS110-001024,ПС 110 кВ Потаскуй,ПС 110 кВ Потаскуй,Substation
2,01628ca3-8af1-4d26-84c3-52ee6c0f1adc,PS110-000253,ПС 110 кВ Пригородная,ПС 110 кВ Пригородная,Substation
3,018c4cfb-1f78-41e9-8e40-001a41114299,PS110-000519,ПС 110 кВ Городская,ПС 110 кВ Городская,Substation
4,0193398d-8d84-40d9-8520-2f8ea56bf5e9,PS110-000294,ПС 110 кВ Коробейниково,ПС 110 кВ Коробейниково,Substation


# ?

In [18]:
# print(sorted(df_rs['name_norm'].unique()))

In [19]:
# print(sorted(df_supa['name_norm'].unique()))

In [20]:
# check = 'южная'
# display(df_rs[df_rs['name_norm'].str.contains(check)])
# df_supa[df_supa['name_norm'].str.contains(check)]

# + сравнение с сопоставленными ранее

In [21]:
uids_unmatched = set(df_rs['Uid'])
df_old_matched = pd.read_csv('old_results/0_результат_ПС_сопоставленные.csv')
uids_old_matched = set(df_old_matched['Uid'])

res = []

uids_common_now_before = uids_unmatched & uids_old_matched
for uid in uids_common_now_before:
    techplace = df_old_matched[df_old_matched['Uid'] == uid]['Техместо'].values[0]
    controller = df_supa[df_supa['Техместо'] == techplace]['Дисп.наименование'].values
    if controller.size == 0:
        controller = ['']
    res.append((
        uid,
        techplace,
        df_rs[df_rs['Uid'] == uid]['name'].values[0], 
        controller[0],
        df_rs[df_rs['Uid'] == uid]['Класс'].values[0]
    ))

df_handy_res = pd.DataFrame(data=res, columns=['Uid', 'Техместо', 'Наименование', 'Диспетчерское наименование', 'Класс'])
techplaces_common_handy_res_union = set(df_handy_res['Техместо']) & set(df_union['Техместо'])
df_handy_res = df_handy_res[~df_handy_res['Техместо'].isin(techplaces_common_handy_res_union)]
display(df_handy_res.head())
display(df_handy_res.info())

,Uid,Техместо,Наименование,Диспетчерское наименование,Класс
0,56984ea4-5a96-436b-8428-56180554b1c6,PS110-000168,ПС 110 кВ Алей,ПС 110 кВ Алей,Substation
1,636e855d-298b-4aff-bc05-916886518918,PS110-000581,ПС 110 кВ Калачинская,ПС 110 кВ Калачинская,Substation
2,0ad916d9-1268-4df8-867d-5f3f747d643d,PS110-000466,ПС 110 кВ Анжерская,ПС 110 кВ Анжерская,Substation
3,8ebce62b-2ee8-44bd-876c-02977962ae9a,PS110-000599,ПС 110 кВ Стрела,ПС 110 кВ Стрела,Substation
4,89b8870e-ccc1-4328-aee0-04fb0958724e,PS110-000432,ПС 110 кВ Н.Березовская,ПС 110 кВ Новоберёзовская,Substation


<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Uid                         39 non-null     object
 1   Техместо                    39 non-null     object
 2   Наименование                39 non-null     object
 3   Диспетчерское наименование  39 non-null     object
 4   Класс                       39 non-null     object
dtypes: object(5)
memory usage: 1.8+ KB


None

In [22]:
df_union_handy = pd.concat([df_union, df_handy_res])
for col in df_union_handy.columns:
    print(col, df_union_handy[col].duplicated().sum())
print(df_union_handy.shape)
display(df_union_handy.info())
display(df_union_handy.head())

# filename = 'output_data/Substation/Substation_result.csv'
# df_union_handy.to_csv(filename, index=False)
# print(f'test saved {filename}')
# display(pd.read_csv(filename).head())
# pd.read_csv(filename).info()

Uid 0
Техместо 0
Наименование 50
Диспетчерское наименование 55
Класс 819
(820, 5)
<class 'pandas.core.frame.DataFrame'>
Index: 820 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Uid                         820 non-null    object
 1   Техместо                    820 non-null    object
 2   Наименование                820 non-null    object
 3   Диспетчерское наименование  817 non-null    object
 4   Класс                       820 non-null    object
dtypes: object(5)
memory usage: 38.4+ KB


None

,Uid,Техместо,Наименование,Диспетчерское наименование,Класс
0,000fe64c-7b85-4bea-9ada-3e514d968485,PS110-000516,ПС 110 кВ Уропская,ПС 110 кВ Уропская,Substation
1,011f18d7-614e-4945-8bf8-81964a8c6a9f,PS110-001024,ПС 110 кВ Потаскуй,ПС 110 кВ Потаскуй,Substation
2,01628ca3-8af1-4d26-84c3-52ee6c0f1adc,PS110-000253,ПС 110 кВ Пригородная,ПС 110 кВ Пригородная,Substation
3,018c4cfb-1f78-41e9-8e40-001a41114299,PS110-000519,ПС 110 кВ Городская,ПС 110 кВ Городская,Substation
4,0193398d-8d84-40d9-8520-2f8ea56bf5e9,PS110-000294,ПС 110 кВ Коробейниково,ПС 110 кВ Коробейниково,Substation


In [23]:
df_supa = df_supa[~df_supa['Техместо'].isin(df_union_handy['Техместо'].unique())]
df_rs = df_rs[~df_rs['Uid'].isin(df_union_handy['Uid'].unique())]
print(df_supa.shape[0], df_rs.shape[0])

# filename = 'output_data/Substation/Substation4preparator_unmatched.csv'
# df_rs.to_csv(filename, index=False)
# print(f'test saved {filename}')
# display(pd.read_csv(filename).head())
# pd.read_csv(filename).info()

368 16


# Оставшиеся

In [24]:
df_rs

,Uid,name,Класс,Substations,name_norm,name_region
191,2fe937ca-34e8-42ba-9281-f4082e142d1e,ПС 110 кВ Северная,Substation,Красноярский край,северная,северная_Красноярский край
238,3c57f6cd-2139-4bc4-93e9-344ddeb37372,ПС 110 кВ Золотая звезда,Substation,Красноярский край,золотая-звезда,золотая-звезда_Красноярский край
261,42f00011-7e42-4f4d-8b54-277c835f17bc,ПС 110 кВ Угольная,Substation,Забайкальский край,угольная,угольная_Забайкальский край
340,55d55f67-0c54-4f6d-aeb5-1b10677ed32e,ПС 110 кВ ГПП-2 АПОВ,Substation,Республика Хакасия,г-два,гпп-два Республика Хакасия
405,69c319a1-7550-495f-b678-3aad4544bf5c,ПС 110 кВ БСМИ,Substation,Красноярский край,бсми,бсми_Красноярский край
466,7be2b9f3-9c42-42df-b3a0-b374cc2e1ad8,ПС 110 кВ Новая Пойма,Substation,Красноярский край,новая-пойма,новая-пойма_Красноярский край
468,7c5f0b82-36bb-4eed-9b1a-1323a47eceb1,ПС 110 кВ БНС ТЭЦ-3,Substation,Красноярский край,тэц-три,тэц-три_Красноярский край
513,89411ab7-28c3-422d-8bab-7238fe271e51,ПС 110 кВ КТПБ-145 ТЭЦ-3,Substation,Красноярский край,ктпб-один,ктпб-один_Красноярский край
518,8a05b3d5-f4df-4da3-8104-955012a544dd,ПС 110 кВ Веселая Гора,Substation,Красноярский край,веселая-гора,веселая-гора_Красноярский край
549,95565592-f3d8-45f0-9e83-0231b8f5f865,ПС 110 кВ Автосборочная,Substation,Забайкальский край,автосборочная,автосборочная_Забайкальский край


In [25]:
# ручное
df_rs.loc[(df_rs['name'] == 'ПС 110 кВ БНС ТЭЦ-3') & (df_rs['Substations'] == 'Красноярский край'), 'name_region'] = 'бнс-тэц-три_Красноярский край'
df_supa.loc[(df_supa['Название'] == 'ПС 110 кВ БНС ТЭЦ-3') & (df_supa['region'] == 'Красноярский край'), 'name_region'] = 'бнс-тэц-три_Красноярский край'
df_supa.loc[(df_supa['Название'] == 'СЗССМ ПС Карат (Ведомственая)') & (df_supa['region'] == 'Республика Хакасия'), 'name_region'] = 'карат_Республика Хакасия'
# ---

supa_name_region_list = df_supa['name_region'].tolist()
rs_name_region_list = df_rs['name_region'].tolist()

supa_name_region = set(supa_name_region_list)
rs_name_region = set(rs_name_region_list)

print('supa_name_region & rs_name_region =', len(supa_name_region & rs_name_region))

supa_name_region_cnt = dict(Counter(supa_name_region_list))
rs_name_region_cnt = dict(Counter(rs_name_region_list))

supa_name_region_single = set([key for key in supa_name_region_cnt.keys() if supa_name_region_cnt[key] == 1])
rs_name_region_single = set([key for key in rs_name_region_cnt.keys() if rs_name_region_cnt[key] == 1])
print('supa_name_region_single & rs_name_region_single', len(supa_name_region_single & rs_name_region_single))

supa_name_region & rs_name_region = 3
supa_name_region_single & rs_name_region_single 3


In [26]:
df_merge3 = df_rs[df_rs['name_region'].isin(rs_name_region_single)].merge(df_supa[df_supa['name_region'].isin(supa_name_region_single)], on='name_region', suffixes=['_rs', '_supa'])
df_merge3.shape[0]

3

In [27]:
for col in df_merge3.columns:
    print(col, df_merge3[col].duplicated().sum())
    assert df_merge3[col].duplicated().sum() == 0 if col in ['Uid', 'Техместо'] else True

Uid 0
name 0
Класс_rs 2
Substations 1
name_norm_rs 0
name_region 0
ГруппПолномочий 1
Техместо 0
Название 0
Дисп.наименование 0
Местоположение 1
НаименованиеМестоположения 1
ВерхнееТМ 0
Район 2
Класс_supa 2
БЕ 1
ПланирующЗавод 1
Видобъекта 2
ОснСредство 0
ПользовСтатус 2
СистемнСтатус 2
Вэксплуат.с 2
Годвыпуска 2
Инвентарн№ 0
НомерОСверх.ТМ 0
НомерОСинв.об. 2
region 1
name_norm_supa 0


In [28]:
needed_cols = ['Uid', 'Техместо', 'name', 'Дисп.наименование', 'Класс_rs']
columns2rename = {'Класс_rs': 'Класс', 
                 'name': 'Наименование',
                 'Дисп.наименование': 'Диспетчерское наименование'}
df_union_handy_merge3 = pd.concat([df_union_handy, df_merge3[needed_cols].rename(columns=columns2rename)])
for col in df_union_handy_merge3.columns:
    print(col, df_union_handy_merge3[col].duplicated().sum())
print(df_union_handy_merge3.shape)
display(df_union_handy_merge3.info())
display(df_union_handy_merge3.head())

filename = 'output_data/Substation/Substation_result.csv'
df_union_handy_merge3.to_csv(filename, index=False)
print(f'test saved {filename}')
display(pd.read_csv(filename).head())
pd.read_csv(filename).info()

Uid 0
Техместо 0
Наименование 50
Диспетчерское наименование 56
Класс 822
(823, 5)
<class 'pandas.core.frame.DataFrame'>
Index: 823 entries, 0 to 2
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Uid                         823 non-null    object
 1   Техместо                    823 non-null    object
 2   Наименование                823 non-null    object
 3   Диспетчерское наименование  819 non-null    object
 4   Класс                       823 non-null    object
dtypes: object(5)
memory usage: 38.6+ KB


None

,Uid,Техместо,Наименование,Диспетчерское наименование,Класс
0,000fe64c-7b85-4bea-9ada-3e514d968485,PS110-000516,ПС 110 кВ Уропская,ПС 110 кВ Уропская,Substation
1,011f18d7-614e-4945-8bf8-81964a8c6a9f,PS110-001024,ПС 110 кВ Потаскуй,ПС 110 кВ Потаскуй,Substation
2,01628ca3-8af1-4d26-84c3-52ee6c0f1adc,PS110-000253,ПС 110 кВ Пригородная,ПС 110 кВ Пригородная,Substation
3,018c4cfb-1f78-41e9-8e40-001a41114299,PS110-000519,ПС 110 кВ Городская,ПС 110 кВ Городская,Substation
4,0193398d-8d84-40d9-8520-2f8ea56bf5e9,PS110-000294,ПС 110 кВ Коробейниково,ПС 110 кВ Коробейниково,Substation


test saved output_data/Substation/Substation_result.csv


,Uid,Техместо,Наименование,Диспетчерское наименование,Класс
0,000fe64c-7b85-4bea-9ada-3e514d968485,PS110-000516,ПС 110 кВ Уропская,ПС 110 кВ Уропская,Substation
1,011f18d7-614e-4945-8bf8-81964a8c6a9f,PS110-001024,ПС 110 кВ Потаскуй,ПС 110 кВ Потаскуй,Substation
2,01628ca3-8af1-4d26-84c3-52ee6c0f1adc,PS110-000253,ПС 110 кВ Пригородная,ПС 110 кВ Пригородная,Substation
3,018c4cfb-1f78-41e9-8e40-001a41114299,PS110-000519,ПС 110 кВ Городская,ПС 110 кВ Городская,Substation
4,0193398d-8d84-40d9-8520-2f8ea56bf5e9,PS110-000294,ПС 110 кВ Коробейниково,ПС 110 кВ Коробейниково,Substation


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823 entries, 0 to 822
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Uid                         823 non-null    object
 1   Техместо                    823 non-null    object
 2   Наименование                823 non-null    object
 3   Диспетчерское наименование  819 non-null    object
 4   Класс                       823 non-null    object
dtypes: object(5)
memory usage: 32.3+ KB


In [29]:
df_supa = df_supa[~df_supa['Техместо'].isin(df_union_handy_merge3['Техместо'].unique())]
df_rs = df_rs[~df_rs['Uid'].isin(df_union_handy_merge3['Uid'].unique())]
print(df_supa.shape[0], df_rs.shape[0])

filename = 'output_data/Substation/Substation4preparator_unmatched.csv'
df_rs.to_csv(filename, index=False)
print(f'test saved {filename}')
display(pd.read_csv(filename).head())
pd.read_csv(filename).info()

365 13
test saved output_data/Substation/Substation4preparator_unmatched.csv


,Uid,name,Класс,Substations,name_norm,name_region
0,2fe937ca-34e8-42ba-9281-f4082e142d1e,ПС 110 кВ Северная,Substation,Красноярский край,северная,северная_Красноярский край
1,3c57f6cd-2139-4bc4-93e9-344ddeb37372,ПС 110 кВ Золотая звезда,Substation,Красноярский край,золотая-звезда,золотая-звезда_Красноярский край
2,42f00011-7e42-4f4d-8b54-277c835f17bc,ПС 110 кВ Угольная,Substation,Забайкальский край,угольная,угольная_Забайкальский край
3,69c319a1-7550-495f-b678-3aad4544bf5c,ПС 110 кВ БСМИ,Substation,Красноярский край,бсми,бсми_Красноярский край
4,7be2b9f3-9c42-42df-b3a0-b374cc2e1ad8,ПС 110 кВ Новая Пойма,Substation,Красноярский край,новая-пойма,новая-пойма_Красноярский край


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Uid          13 non-null     object
 1   name         13 non-null     object
 2   Класс        13 non-null     object
 3   Substations  13 non-null     object
 4   name_norm    13 non-null     object
 5   name_region  13 non-null     object
dtypes: object(6)
memory usage: 756.0+ bytes
